In [1]:
import numpy as np
import igraph as ig
from scipy.special import betaln

Data Generation

In [39]:
theta = np.array([[0.75, 0.25, 0.1, 0.9],
                  [0.1, 0.75, 0.1, 0.1],
                  [0.1, 0.75, 0.25, 0.9]])

In [40]:
Sr = [20, 20, 20]
Sc = [15, 15, 15, 15]

In [41]:
def generate_graph_bicluster(Sr, Sc, thetas):
    # Kr = thetas.shape[0]
    # Kc = thetas.shape[1]

    N = np.sum(Sr)

    Hr = []
    c = 0
    for i in Sr:
        for j in range(i):
            Hr += [c]
        c+=1

    Hc = []
    c = 0
    for i in Sc:
        for j in range(i):
            Hc += [c]
        c+=1

    X = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            if i == j:
                continue
            X[i, j] = np.random.binomial(1, thetas[Hr[i], Hc[j]])

    return X


In [42]:
X = generate_graph_bicluster(Sr, Sc, theta)

In [23]:
def irm_bicluster(X, T, a, b, A, set_seed = True, random_seed = 42, print_iter = False):
    N = len(X)
    zr = np.ones([N, 1])
    zc = np.ones([N, 1])
    Z = []
    Zr = []
    Zc = []

    if set_seed:
        np.random.seed(random_seed)

    for t in range(T):
        for n in range(N):
            nn = [_ for _ in range(N)]
            nn.remove(n)

            X_ = X.copy().astype(int)
            X_[n,:] = 0

            Kr = len(zr[0])

            if Kr > 1:
                idx = np.argwhere(np.sum(zr[nn], 0) == 0)
                zr = np.delete(zr, idx, axis=1)
                Kr -= len(idx)

            # m = n. of nodes in each component 
            mr = np.sum(zr[nn,:], 0)[np.newaxis] #newaxis allows m to become 2d array (for transposing)
            mc = np.sum(zc[nn,:], 0)[np.newaxis]
            Mc = np.tile(mc, (Kr, 1))
            M1 = zr.T @ X_ @ zc

            X_rev = (np.where((X_==0)|(X_==1), X_^1, X_) - np.eye(X_.shape[0])).copy() #reverse matrix for non_links
            X_rev[n,:] = 0
            M0 = zr.T @ X_rev @ zc #n. of non-links between biclusters without current node

            r = zc[nn,:].T @ X[n, nn]
            R = np.tile(r, (Kr, 1))

            logLik_exComp = np.sum(betaln(M1+R+a, M0+Mc-R+b) - betaln(M1+a, M0+b),1)
            logLik_newComp = np.sum(betaln(r+a, mc-r+b) - betaln(a,b),1)

            logLik = np.concatenate([logLik_exComp, logLik_newComp])
            logPrior = np.log(np.append(mr, A))

            logPost = logPrior + logLik

            P = np.exp(logPost-max(logPost)) 

            # Assignment through random draw fron unif(0,1), taking first value from prob. vector
            draw = np.random.rand()
            i = np.argwhere(draw<np.cumsum(P)/sum(P))[0]

            zr[n,:] = 0
            if i == Kr: # If new component: add new column to partition matrix
                zr = np.hstack((zr, np.zeros((N,1)))) 
            zr[n,i] = 1

        for n in range(N):
            nn = [_ for _ in range(N)]
            nn.remove(n)

            X_ = X.copy().astype(int)
            X_[:,n] = 0

            Kc = len(zc[0])

            if Kc > 1:
                idx = np.argwhere(np.sum(zc[nn], 0) == 0)
                zc = np.delete(zc, idx, axis=1)
                Kc -= len(idx)

            # m = n. of nodes in each component 
            mr = np.sum(zr[nn,:], 0)[np.newaxis] #newaxis allows m to become 2d array (for transposing)
            mc = np.sum(zc[nn,:], 0)[np.newaxis]
            Mr = np.tile(mr.T, (1, Kc))

            M1 = zr.T @ X_ @ zc

            X_rev = (np.where((X_==0)|(X_==1), X_^1, X_) - np.eye(X_.shape[0])).copy() #reverse matrix for non_links
            X_rev[:,0] = 0
            M0 = zr.T @ X_rev @ zc #n. of non-links between biclusters without current node

            s = zr[nn,:].T @ X[nn, n]
            S = np.tile(s[np.newaxis].T, (1, Kc))

            logLik_exComp = np.sum(betaln(M1+S+a, M0+Mr-S+b) - betaln(M1+a, M0+b), 0)
            logLik_newComp = np.sum(betaln(s+a, mr-s+b) - betaln(a,b),1)

            logLik = np.concatenate([logLik_exComp, logLik_newComp])
            logPrior = np.log(np.append(mc, A))

            logPost = logPrior + logLik

            P = np.exp(logPost-max(logPost)) 

            # Assignment through random draw fron unif(0,1), taking first value from prob. vector
            draw = np.random.rand()
            i = np.argwhere(draw<np.cumsum(P)/sum(P))[0]

            zc[n,:] = 0
            if i == Kc: # If new component: add new column to partition matrix
                zc = np.hstack((zc, np.zeros((N,1)))) 
            zc[n,i] = 1

        Z.append([zr.copy(), zc.copy()])
        Zr.append(zr.copy())
        Zc.append(zc.copy())
    
    return [Zr, Zc]

In [44]:
X

array([[0., 1., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 1., 0., ..., 1., 1., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [19]:
T = 500
a = 1
b = 1
A = 1

In [24]:
Z = irm_bicluster(X, T , a, b, A, set_seed = True, random_seed = 42)

In [47]:
t = Z[-1]

In [26]:
len(Z[0])

500

In [11]:
#test X
X = np.array([[0, 1, 0, 0, 1],
              [0, 0, 1, 1, 0],
              [0, 1, 0, 1, 1],
              [1, 1, 0, 0, 1],
              [0, 1, 0, 1, 0]])

zr = np.array([[1, 0],
               [1, 0],
               [1, 0],
               [0, 1],
               [0, 1]])

zc = np.array([[1, 0, 0],
               [1, 0, 0],
               [0, 1, 0],
               [0, 1, 0],
               [0, 0, 1]])

a = 1
b = 1
A = 1

In [4]:
Kr = len(zr[0])
Kc = len(zc[0])

mr = np.sum(zr, 0)[np.newaxis] #newaxis allows m to become 2d array (for transposing)
mc = np.sum(zc, 0)[np.newaxis]

In [9]:
M1 = zr.T @ X @ zc

X_rev = (np.where((X==0)|(X==1), X^1, X) - np.eye(X.shape[0])).copy() #reverse matrix for non_links
M0 = zr.T @ X_rev @ zc

In [13]:
np.sum(betaln(M1 + a, M0 + b) - betaln(a, b))

-16.15423469757419

In [32]:
N = len(X)
# zr = np.ones([N, 1])
# zc = np.ones([N, 1])
Zr = []
Zc = []

In [55]:
# for t in range(T):
#     for n in range(N):
n = 0

nn = [_ for _ in range(N)]
nn.remove(n)

X_ = X[np.ix_(nn,nn)]

Kr = len(zr[0])
Kc = len(zc[0])

if Kr > 1:
    idx = np.argwhere(np.sum(zr[nn], 0) == 0)
    zr = np.delete(zr, idx, axis=1)
    Kr -= len(idx)

# m = n. of nodes in each component 
mr = np.sum(zr[nn,:], 0)[np.newaxis] #newaxis allows m to become 2d array (for transposing)
mc = np.sum(zc[nn,:], 0)[np.newaxis]

# M = max links from other clusts
# M = np.tile(m, (Kr, 1))

M1 = zr[nn,:].T @ X_ @ zc[nn,:]

X_rev = np.where((X_==0)|(X_==1), X_^1, X_) - np.eye(X_.shape[0]) #reverse matrix for non_links
M0 = zr[nn,:].T @ X_rev @ zc[nn,:] #n. of non-links between biclusters without current node

r = zc[nn,:].T @ X[n, nn]
R = np.tile(r, (Kr, 1))

s = zr[nn,:].T @ X[nn, n]
S = np.tile(s[np.newaxis].T, (1, Kc))



In [37]:
from igraph.clustering import compare_communities
from scipy.special import betaln
from eval import retrieve_samples

In [34]:
def compute_VIs(C):
    N = len(C)
    VIs = np.zeros((N,N))

    for i in range(N):
        for j in range(i):
            VIs[i,j] = compare_communities(C[i], C[j])

    VIs += VIs.T

    return VIs

def compute_edge_quantities(X, zr, zc = None, mode = 'biclustering'):
    if mode == 'biclustering':
        mr = np.sum(zr, 0)[np.newaxis] #newaxis allows m to become 2d array (for transposing)
        mc = np.sum(zc, 0)[np.newaxis]

        M1 = zr.T @ X @ zc

        X_rev = (np.where((X==0)|(X==1), X^1, X) - np.eye(X.shape[0])).copy() #reverse matrix for non_links
        M0 = zr.T @ X_rev @ zc

    return M1, M0

# def compute_logL(X, zr, zc, a=1, b=1):
#     M1, M0 = compute_edge_quantities(X, zr, zc)

def compute_logL_bicl(X, zr, zc, a=1, b=1):
    M1, M0 = compute_edge_quantities(X, zr, zc, mode = 'biclustering')
    logLlhood = np.sum(betaln(M1 + a, M0 + b) - betaln(a, b))
    return logLlhood



def compute_logLs(X, Z, mode = 'biclustering'):
    
    if mode == 'biclustering':
        N = len(Z[0])
        logLs = [0]*N
        for i in range(N):
            logLs[i] = compute_logL_bicl(X, Z[0][i], Z[1][i])

    return logLs
    

In [83]:
Zr = retrieve_samples(Z[0], gap = 10)
Zc = retrieve_samples(Z[1], gap = 10)

Cr = [np.where(x == 1)[1].tolist() for x in Zr]
Cc = [np.where(y == 1)[1].tolist() for y in Zc]

logLs = compute_logLs(X, [Zr, Zc], mode = 'biclustering')

VIr = compute_VIs(Cr)
VIc = compute_VIs(Cc)

VI = (VIr + VIc)/2
VI = VI.sum(0)/VI.shape[0]
VI *= logLs

idx_min = np.argmin(VI) 
z_hat = [Zr[idx_min], Zc[idx_min]]
c_hat = [np.where(z_hat[0]==1)[1], np.where(z_hat[1]==1)[1]]

In [84]:
c_hat

[array([0, 1, 1, 2, 0], dtype=int64), array([1, 1, 0, 0, 0], dtype=int64)]

In [79]:
z_hat = [Zr[idx_min], Zc[idx_min]]

In [81]:
z_hat

[array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.]]),
 array([[0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.]])]

In [82]:
np.where(z_hat[0]==1)[1]

array([0, 1, 1, 2, 0], dtype=int64)

In [65]:
logLs

[-12.712215321391783,
 -12.619505923287514,
 -12.021668922531894,
 -14.252660362338933,
 -11.585432155756976,
 -12.550513051800563,
 -12.460900893110876,
 -11.749735207048253,
 -12.021668922531894,
 -13.628506053265937,
 -14.727118341934046,
 -15.79755975363546,
 -14.727118341934046,
 -15.35572700135642,
 -12.396362371973304,
 -12.907187995739296,
 -13.089509552533249,
 -15.173816882543996,
 -14.970881180450993,
 -14.950261893248255,
 -12.801827480081469,
 -11.143599403477936,
 -12.668296087456948,
 -13.851649604580146,
 -12.396362371973305]

In [61]:
VI *= logLs

In [62]:
VI

array([[ -0.        , -10.06780156,  -8.33278592, -10.62495915,
         -5.81812212,  -6.30277892,  -8.86583364,  -6.51543666,
         -8.33278592,  -6.38104538, -11.74924802, -13.42984715,
         -7.6660238 , -12.25075001, -12.25691643, -12.76199581,
         -9.7578628 , -15.62376629, -13.23577748, -13.99981761,
         -9.63202   ,  -5.21757947,  -9.53155176,  -9.13055548,
         -8.17127762],
       [-10.14176482,  -0.        ,  -6.66622874, -16.55247396,
         -7.42420405, -11.52239057, -10.5932813 ,  -8.14429583,
         -6.66622874, -15.82760592,  -9.70763591, -15.61985395,
        -13.79086013, -22.89452888, -10.53841571, -14.551312  ,
         -7.94327147, -15.10816291, -15.31118229, -11.92727124,
         -7.85730988,  -6.76241038,  -7.77535302, -14.89129461,
         -1.71850073],
       [ -8.81143621,  -6.99773996,  -0.        , -11.10926181,
         -5.42445137,  -5.87631491,  -4.53032867,  -1.62885917,
         -0.        , -12.0489817 , -13.79086013, -15.6198

In [162]:
# for t in range(T):
#     for n in range(N):
a = 1
b = 1
A = 1

N = len(X)
for i in range(N):
    nn = [_ for _ in range(N)]
    nn.remove(n)

    # X_ = X[np.ix_(nn,nn)]

    X_ = X.copy()
    X_[n,:] = 0

    Kr = len(zr[0])
    Kc = len(zc[0])

    if Kr > 1:
        idx = np.argwhere(np.sum(zr[nn], 0) == 0)
        zr = np.delete(zr, idx, axis=1)
        Kr -= len(idx)

    # m = n. of nodes in each component 
    mr = np.sum(zr[nn,:], 0)[np.newaxis] #newaxis allows m to become 2d array (for transposing)
    mc = np.sum(zc[nn,:], 0)[np.newaxis]
    Mc = np.tile(mc, (Kr, 1))

    # M = max links from other clusts
    # M = np.tile(m, (Kr, 1))

    M1 = zr.T @ X_ @ zc

    X_rev = (np.where((X_==0)|(X_==1), X_^1, X_) - np.eye(X_.shape[0])).copy() #reverse matrix for non_links
    X_rev[n,:] = 0
    M0 = zr.T @ X_rev @ zc #n. of non-links between biclusters without current node

    r = zc[nn,:].T @ X[n, nn]
    R = np.tile(r, (Kr, 1))

    logLik_exComp = np.sum(betaln(M1+R+a, M0+Mc-R+b) - betaln(M1+a, M0+b),1)
    logLik_newComp = np.sum(betaln(r+a, mc-r+b) - betaln(a,b),1)

    logLik = np.concatenate([logLik_exComp, logLik_newcomp])
    logPrior = np.log(np.append(mr, A))

    logPost = logPrior + logLik

    P = np.exp(logPost-max(logPost)) 

    # Assignment through random draw fron unif(0,1), taking first value from prob. vector
    draw = np.random.rand()
    i = np.argwhere(draw<np.cumsum(P)/sum(P))[0]

    zr[n,:] = 0
    if i == Kr: # If new component: add new column to partition matrix
        zr = np.hstack((zr, np.zeros((N,1)))) 
    zr[n,i] = 1


In [16]:
N = len(X)

zr = np.ones([N, 1])
zc = np.ones([N, 1])

In [17]:
n = 0

nn = [_ for _ in range(N)]
nn.remove(n)

# X_ = X[np.ix_(nn,nn)]

X_ = X.copy()
X_[:,n] = 0

Kr = len(zr[0])
Kc = len(zc[0])

In [20]:
mr = np.sum(zr[nn,:], 0)[np.newaxis] #newaxis allows m to become 2d array (for transposing)
mc = np.sum(zc[nn,:], 0)[np.newaxis]
Mc = np.tile(mc, (Kr, 1))

M1 = zr.T @ X_ @ zc

In [32]:
X_rev = (np.where((X_==0)|(X_==1), X_^1, X_) - np.eye(X_.shape[0])).copy()

In [31]:
X_ = X_.astype(int)

In [33]:
n = 0

nn = [_ for _ in range(N)]
nn.remove(n)

# X_ = X[np.ix_(nn,nn)]

X_ = X.copy().astype(int)
X_[:,n] = 0

Kr = len(zr[0])
Kc = len(zc[0])

if Kr > 1:
    idx = np.argwhere(np.sum(zr[nn], 0) == 0)
    zr = np.delete(zr, idx, axis=1)
    Kr -= len(idx)

# m = n. of nodes in each component 
mr = np.sum(zr[nn,:], 0)[np.newaxis] #newaxis allows m to become 2d array (for transposing)
mc = np.sum(zc[nn,:], 0)[np.newaxis]
Mc = np.tile(mc, (Kr, 1))

# M = max links from other clusts
# M = np.tile(m, (Kr, 1))

M1 = zr.T @ X_ @ zc

X_rev = (np.where((X_==0)|(X_==1), X_^1, X_) - np.eye(X_.shape[0])).copy() #reverse matrix for non_links
X_rev[n,:] = 0
M0 = zr.T @ X_rev @ zc #n. of non-links between biclusters without current node

r = zc[nn,:].T @ X[n, nn]
R = np.tile(r, (Kr, 1))

logLik_exComp = np.sum(betaln(M1+R+a, M0+Mc-R+b) - betaln(M1+a, M0+b),1)
logLik_newComp = np.sum(betaln(r+a, mc-r+b) - betaln(a,b),1)

logLik = np.concatenate([logLik_exComp, logLik_newComp])
logPrior = np.log(np.append(mr, A))

logPost = logPrior + logLik

P = np.exp(logPost-max(logPost)) 

# Assignment through random draw fron unif(0,1), taking first value from prob. vector
draw = np.random.rand()
i = np.argwhere(draw<np.cumsum(P)/sum(P))[0]

zr[n,:] = 0
if i == Kr: # If new component: add new column to partition matrix
    zr = np.hstack((zr, np.zeros((N,1)))) 
zr[n,i] = 1

NameError: name 'logLik_newcomp' is not defined

In [297]:
for n in range(N):
    nn = [_ for _ in range(N)]
    nn.remove(n)

    # X_ = X[np.ix_(nn,nn)]

    X_ = X.copy()
    X_[:,n] = 0

    Kr = len(zr[0])
    Kc = len(zc[0])

    if Kc > 1:
        idx = np.argwhere(np.sum(zc[nn], 0) == 0)
        zc = np.delete(zc, idx, axis=1)
        Kc -= len(idx)

    # m = n. of nodes in each component 
    mr = np.sum(zr[nn,:], 0)[np.newaxis] #newaxis allows m to become 2d array (for transposing)
    mc = np.sum(zc[nn,:], 0)[np.newaxis]
    Mr = np.tile(mr.T, (1, Kc))

    M1 = zr.T @ X_ @ zc

    X_rev = (np.where((X_==0)|(X_==1), X_^1, X_) - np.eye(X_.shape[0])).copy() #reverse matrix for non_links
    X_rev[:,0] = 0
    M0 = zr.T @ X_rev @ zc #n. of non-links between biclusters without current node

    s = zr[nn,:].T @ X[nn, n]
    S = np.tile(s[np.newaxis].T, (1, Kc))

    logLik_exComp = np.sum(betaln(M1+S+a, M0+Mr-S+b) - betaln(M1+a, M0+b), 0)
    logLik_newComp = np.sum(betaln(s+a, mr-s+b) - betaln(a,b),1)

    logLik = np.concatenate([logLik_exComp, logLik_newComp])
    logPrior = np.log(np.append(mc, A))

    logPost = logPrior + logLik

    P = np.exp(logPost-max(logPost)) 

    # Assignment through random draw fron unif(0,1), taking first value from prob. vector
    draw = np.random.rand()
    i = np.argwhere(draw<np.cumsum(P)/sum(P))[0]

    zc[n,:] = 0
    if i == Kc: # If new component: add new column to partition matrix
        zc = np.hstack((zc, np.zeros((N,1)))) 
    zc[n,i] = 1

In [298]:
zc

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [268]:
np.sum(betaln(M1+S+a, M0+Mr-S+b) - betaln(M1+a, M0+b), 0)

array([-4.19970508, -3.14988295, -3.40119738])

In [271]:
np.sum(betaln(s+a, mr-s+b) - betaln(a,b), 1)

array([-2.89037176])

In [289]:
zc[n,:] = 0
if i == Kc: # If new component: add new column to partition matrix
    zc = np.hstack((zc, np.zeros((N,1)))) 
zc[n,i] = 1

In [290]:
zc

array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

In [85]:
#test X
X = np.array([[0, 1, 0, 0, 1],
              [0, 0, 1, 1, 0],
              [0, 1, 0, 1, 1],
              [1, 1, 0, 0, 1],
              [0, 1, 0, 1, 0]])

zr = np.array([[1, 0],
               [1, 0],
               [1, 0],
               [0, 1],
               [0, 1]])

# zc = np.array([[1, 0, 0],
#                [1, 0, 0],
#                [0, 1, 0],
#                [0, 1, 0],
#                [0, 0, 1]])

a = 1
b = 1
A = 1

In [86]:
M1 = zr.T @ X @ zr

In [87]:
M1

array([[3, 4],
       [3, 2]])

In [88]:
X = X.copy().astype(int)
X_rev = (np.where((X==0)|(X==1), X^1, X) - np.eye(X.shape[0])).copy() #reverse matrix for non_links
M0 = zr.T @ X_rev @ zr

In [89]:
M0

array([[3., 2.],
       [3., 0.]])